# df_points with UNIX Timestamp #
## Creates df_points.csv with all points and idletimes #
## Checks the valid geo location
## deletes the invalid points: 
2019 - ca. 9.5%

2020 - ca. 2.7%

2021 - ca.

### ca 1h for 1 year

In [15]:
def importdata(source):
    df = pd.read_csv(source)
    df = df[['bike_id', 'start_time', 'end_time', 'start_lat', 'start_lng', 'end_lat', 'end_lng']]
    return df

In [16]:
def dicOutOfDf(df01):
    import pandas as pd
    from tqdm import tqdm
    # Dict out of DataFrames of Trips per Bike

    # init DFs for every bike
    myDFs = {}

    for ind in tqdm(df01.index):
        if df01['bike_id'][ind] not in myDFs:
            myDFs[df01['bike_id'][ind]] = pd.DataFrame(
                columns=['bike_id', 'start_time', 'end_time', 'start_lat', 'start_lng', 'end_lat', 'end_lng'])
    return myDFs

In [17]:
def addRentalsToDf(df01,myDFs):
    from tqdm import tqdm
    # Add all rentals to their bike_id DF
    for ind in tqdm(df01.index):
        if df01['bike_id'][ind] in myDFs:
            myDFs[df01['bike_id'][ind]].loc[df01.index[ind]] = df01.iloc[ind]
    return myDFs

In [18]:
def sortDFs(myDFs):
    # sort every DF
    for df in myDFs.values():
        df[['bike_id', 'start_time','end_time']] = df[['bike_id', 'start_time','end_time']].astype(int)
        df[['start_lat','start_lng', 'end_lat', 'end_lng']] = df[['start_lat','start_lng', 'end_lat', 'end_lng']].astype(float)
        df.sort_values(by=['start_time'], inplace=True)
        df.reset_index(drop=True, inplace=True)
    return myDFs

In [19]:
def dfPoints(myDFs,radius):
    # DF of all points with stoodtime
    inval = 0
    val = 0 
    
    df_points = pd.DataFrame(columns=['bike_id', 'lat', 'lng', 'timestood', 'time_start', 'time_end'])
    df_points[['bike_id', 'timestood', 'time_start', 'time_end']] = df_points[['bike_id', 'timestood', 'time_start', 'time_end']].astype(int)
    df_points[['lng', 'lat']] = df_points[['lng', 'lat']].astype(float)
    
    for df in tqdm(myDFs.values()):
        for ind in df.index[1:]:
            #in SECOUNDS
            timestood = (df['start_time'][ind] - df['end_time'][ind - 1]).astype(int)
            distance = qick_distance(df['end_lat'][ind-1], df['end_lng'][ind-1], df['start_lat'][ind], df['start_lng'][ind])
            if distance > radius:
                inval +=1
                continue
            val +=1
            dict = {'bike_id': df['bike_id'][ind], 'lng': df['start_lng'][ind], 'lat': df['start_lat'][ind],
                    'timestood': timestood, 'time_start': df['end_time'][ind - 1], 'time_end': df['start_time'][ind]}
            df_points = df_points.append(dict, ignore_index=True)
    print(f"invalides: {inval} from {val} thats {inval/val * 100} %")
    return df_points

In [20]:
# returns distance in meter
# source: https://www.it-swarm.com.de/de/python/wie-kann-ich-die-entfernung-zwischen-zwei-punkten-breitengrad-laengengrad-schnell-schaetzen/1072488907/
def qick_distance(Lat1, Long1, Lat2, Long2):
    x = Lat2 - Lat1
    y = (Long2 - Long1)*cos((Lat2 + Lat1)*0.00872664626)  
    return 111.138*sqrt(x*x+y*y)*1000

In [21]:
def keplerMap(df_points):
    heatmap = KeplerGl()
    heatmap.add_data(data=df_points, name='points')
    heatmap.save_to_html(file_name='heatmap_test.html')

# Main 
## Create map

In [24]:
from keplergl import KeplerGl
import pandas as pd
pd.options.mode.chained_assignment = None
from tqdm import tqdm
from geopy.distance import geodesic
import h3

source = '../data/raw_lendings/lendings_2021.csv'

df = importdata(source)
myDfs = dicOutOfDf(df)
myDfs = addRentalsToDf(df, myDfs)
myDfs = sortDFs(myDfs)


100%|██████████| 571894/571894 [14:24<00:00, 661.76it/s]


## Save MyDfs Map to file 

import pickle
with open('saved_myDfs_2019.pkl', 'wb') as f:
    pickle.dump(myDfs, f)

## Create DF_points 
### without invalid points
52523 from 550865

def importMyDFsDic():
    with open('../data/saved_myDfs.pkl', 'rb') as f:
        myDfs = pickle.load(f)
    return myDfs

### ggf Map from file importieren
import pickle
myDf_imported = importMyDFsDic()

In [25]:
# create DF_POints
from math import cos, sqrt
import pandas as pd
pd.options.mode.chained_assignment = None

radius = 20
df_points = dfPoints(myDfs,radius)

100%|██████████| 1203/1203 [39:35<00:00,  1.97s/it]

invalides: 21842 from 548849 thats 3.979600946708475 %


## Save df_points_valid_geov01.csv

df_points.to_csv("../data/df_points_2019_v1.csv",index=False)

# Split Points at midnight

## save flag feature & time_stood_next_day

In [26]:
df_points['timestood'] = pd.to_timedelta(df_points['timestood'],unit='s')
df_points['time_start'] = pd.to_datetime(df_points['time_start'],unit='s')
df_points['time_end'] = pd.to_datetime(df_points['time_end'],unit='s')

In [29]:
from tqdm import tqdm
from datetime import timedelta
import numpy as np

changed = 1
iterations = 0

df_points['next_day_idle_time'] = np.nan

while changed:
    iterations += 1
    changed = 0
    for ind in tqdm(df_points.index):
        ts = df_points['time_start'][ind]
        te = df_points['time_end'][ind]
        t0 = ts.replace(hour=23, minute=59, second=59)
        
        # is timestood greater than same date 23:59:59
        #   -> over midnight
        if ts + df_points['timestood'][ind] > t0:
            changed = 1
            
            underhang = t0 - ts
            overhang = te - t0
            
            # fix current day
            df_points['time_end'][ind] = ts.replace(hour=23, minute=59, second=59)
            df_points['timestood'][ind] = underhang
            df_points['next_day_idle_time'][ind] = overhang
            
            # add new row for the next day
            time_start = ts.replace(hour=0, minute=0, second=0)
            time_start += timedelta(days=1)
            flag = 0
            if time_start + overhang > time_start.replace(hour=23, minute=59, second=59):
                flag = 1
            
            dict = {'bike_id': df_points['bike_id'][ind], 'lng': df_points['lng'][ind], 'lat': df_points['lat'][ind],
                    'timestood': overhang, 'time_start': time_start, 'time_end': te, 'flag' : flag, 'next_day_idle_time': np.nan}
            df_points = df_points.append(dict, ignore_index=True)

print(iterations)

100%|██████████| 744652/744652 [00:35<00:00, 21141.19it/s]

113


In [40]:
df_points.head()

,bike_id,lat,lng,timestood,time_start,time_end,next_day_idle_time,flag,idle_time
0,75838.0,51.331022,12.316226,0 days 00:37:55,2020-12-31 23:22:04,2020-12-31 23:59:59,NaN,NaN,37.916667
1,75838.0,51.326178,12.334760,0 days 04:32:11,2021-01-04 19:27:48,2021-01-04 23:59:59,0 days 05:29:32,NaN,272.183333
2,75838.0,51.332178,12.343082,0 days 07:34:29,2021-01-05 05:34:11,2021-01-05 13:08:40,NaN,NaN,454.483333
3,75838.0,51.324615,12.336922,0 days 04:54:53,2021-01-05 13:13:36,2021-01-05 18:08:29,NaN,NaN,294.883333
4,75838.0,51.330267,12.336166,0 days 05:47:30,2021-01-05 18:12:29,2021-01-05 23:59:59,0 days 07:32:36,NaN,347.500000


In [41]:
from pandas import Timedelta
# timestood to idle_time in MIN
def f(row):
    return row['timestood'].seconds / 60

def v(row):
    if row['next_day_idle_time'] == np.nan:
        return 0
    elif row['next_day_idle_time'].seconds < Timedelta(days=1).seconds:
        return row['next_day_idle_time'].seconds / 60
    else: return 1439

df_points['idle_time'] = df_points.apply(f, axis=1)
#df_points['next_day_idle_time'] = df_points.apply(v, axis=1)
del df_points['timestood']

In [42]:
df_points['idle_time'] = df_points['idle_time'].astype(int)

# Save V2 df_points split

df_points.to_csv("../data/df_points_2019_v2.csv",index=False)

In [43]:
df_points['idle_time'].max()

1439

In [44]:
df_points.head()

,bike_id,lat,lng,time_start,time_end,next_day_idle_time,flag,idle_time
0,75838.0,51.331022,12.316226,2020-12-31 23:22:04,2020-12-31 23:59:59,NaN,NaN,37
1,75838.0,51.326178,12.334760,2021-01-04 19:27:48,2021-01-04 23:59:59,0 days 05:29:32,NaN,272
2,75838.0,51.332178,12.343082,2021-01-05 05:34:11,2021-01-05 13:08:40,NaN,NaN,454
3,75838.0,51.324615,12.336922,2021-01-05 13:13:36,2021-01-05 18:08:29,NaN,NaN,294
4,75838.0,51.330267,12.336166,2021-01-05 18:12:29,2021-01-05 23:59:59,0 days 07:32:36,NaN,347


## Add H3 Index

In [45]:
import h3

#dfh3 = pd.read_csv('../data/df_points_2020_v2.csv')
#dfh3['hex_id'] = 0
#dfh3.head()

df_points['hex_id'] = 0

for ind in tqdm(df_points.index):
    df_points['hex_id'][ind] = h3.geo_to_h3(df_points['lat'][ind], df_points['lng'][ind], 8)
df_points.head()

100%|██████████| 744652/744652 [00:15<00:00, 49533.92it/s]


,bike_id,lat,lng,time_start,time_end,next_day_idle_time,flag,idle_time,hex_id
0,75838.0,51.331022,12.316226,2020-12-31 23:22:04,2020-12-31 23:59:59,NaN,NaN,37,881f1a8dd7fffff
1,75838.0,51.326178,12.334760,2021-01-04 19:27:48,2021-01-04 23:59:59,0 days 05:29:32,NaN,272,881f1a8d91fffff
2,75838.0,51.332178,12.343082,2021-01-05 05:34:11,2021-01-05 13:08:40,NaN,NaN,454,881f1a8d9bfffff
3,75838.0,51.324615,12.336922,2021-01-05 13:13:36,2021-01-05 18:08:29,NaN,NaN,294,881f1a8d91fffff
4,75838.0,51.330267,12.336166,2021-01-05 18:12:29,2021-01-05 23:59:59,0 days 07:32:36,NaN,347,881f1a8d9bfffff


# Add weather Data

In [47]:
dfw = pd.read_csv('../data/weather_data.csv')
dfw['datetime'] = pd.to_datetime(dfw['dt'], unit='s')
dfw = dfw.set_index(['datetime'])
dfw.head()

df_points['temp'] = np.nan
df_points['rain'] = np.nan
df_points['snow'] = np.nan

from tqdm import tqdm
pd.options.mode.chained_assignment = None

for ind in tqdm(df.index):
    wint = df_points['time_start'][ind].replace(minute=0,second=0)
    df_points['temp'][ind] = dfw['temp'][wint].copy()
    df_points['rain'][ind] = dfw['rain_1h'][wint].copy()
    df_points['snow'][ind] = dfw['snow_1h'][wint].copy()

100%|██████████| 571894/571894 [08:28<00:00, 1124.28it/s]


KeyError: 'time_start'

In [49]:
df_points['dt_start'] = pd.to_datetime(df_points['time_start']).map(pd.Timestamp.timestamp).astype(int)
df_points['dt_end'] = pd.to_datetime(df_points['time_end']).map(pd.Timestamp.timestamp).astype(int)
df_points['bike_id'] = df_points['bike_id'].astype(int)

In [56]:
#df_points['idle_time_next_day'] = pd.Timedelta(df_points['next_day_idle_time']).seconds.astype(int)
def x(row):
    tmp = pd.Timedelta(row['next_day_idle_time']).seconds / 60
    if tmp < 1439:
        return tmp
    else: return 1439

df_points['next_day_idle_time'] = df_points.apply(x,axis=1)

KeyError: 'time_start'

In [57]:
del df_points['time_start']
del df_points['time_end']
del df_points['next_day_idle_time']

In [58]:
df_points.head()

,bike_id,lat,lng,flag,idle_time,hex_id,temp,rain,snow,dt_start,dt_end,idle_time_next_day
0,75838,51.331022,12.316226,NaN,37,881f1a8dd7fffff,2.72,NaN,NaN,1609456924,1609459199,1439.000000
1,75838,51.326178,12.334760,NaN,272,881f1a8d91fffff,0.43,NaN,NaN,1609788468,1609804799,329.533333
2,75838,51.332178,12.343082,NaN,454,881f1a8d9bfffff,0.39,NaN,0.44,1609824851,1609852120,1439.000000
3,75838,51.324615,12.336922,NaN,294,881f1a8d91fffff,0.59,NaN,0.26,1609852416,1609870109,1439.000000
4,75838,51.330267,12.336166,NaN,347,881f1a8d9bfffff,0.93,NaN,NaN,1609870349,1609891199,452.600000


# Save

In [32]:
df_points.to_csv('../data/df_points/df_points_2021.csv', index=False)